In [ ]:
from IPython.display import display
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.csg import *
import netgen.gui

In [ ]:
geo = Mesh(unit_square.GenerateMesh(maxh=0.1))
geo.nv, geo.ne   # number of vertices & elements

In [ ]:
scene = Draw(geo)
#clear_output()
#scene.Draw(height="1vh")

In [4]:
order = 1
fes_curl = HCurl(geo, order=order, dirichlet="boundary")  # For 1-forms, H(curl)
fes_H1 = H1(geo, order=order, dirichlet="boundary")     # For 0-forms, H1 space

prodSpc = fes_curl * fes_H1

# Trial and test functions for product space (HCurl and HDiv components)
omega_HCurl, sigma_H1 = prodSpc.TrialFunction() 
eta_HCurl, tau_H1 = prodSpc.TestFunction()

One of the most important concepts for the Thesis, the De'Rahm complex:

(also the concept I need to look up all the time when coding NGSolve for the thesis..)

$$

0 \longrightarrow H\Lambda^0(\Omega) \xrightarrow{d} H\Lambda^1(\Omega) \xrightarrow{d} \dots \xrightarrow{d} H\Lambda^n(\Omega) \longrightarrow 0

$$

$$

0 \longrightarrow H^1(\Omega) \xrightarrow{\text{grad}} H(\text{curl}, \Omega) \xrightarrow{\text{curl}} H(\text{div}, \Omega) \xrightarrow{\text{div}} L^2(\Omega) \longrightarrow 0

$$

- $H^1(\Omega)$ are 0-forms or scalars
- $H(\text{curl}, \Omega)$ are 1-forms or vectorfiels
- $H(\text{div}, \Omega)$ are covector-valued 2-forms
  - Camilos maybe wrong interpretation of "covector-valued 2-forms in 2D":
  - flux-density on infinitesimal patches in a subset of $\mathbb{R}^2$

In [5]:
# 3. Bilinear forms for H1 and HCurl spaces - Equation (3a)

# H1 part (0-forms)
a = BilinearForm(prodSpc)
a += sigma_H1 * tau_H1 * dx

#HCurl part (1-forms)
a += -omega_HCurl * grad(tau_H1) * dx

In [6]:
# 4. Easy part for equation (3b), the inner products without the boundary parts
a += grad(sigma_H1) * eta_HCurl * dx
a += curl(omega_HCurl) * curl(eta_HCurl) * dx

Hard part for (3b): The boundary terms, the first one being:

$$
.. + \int_{\partial \Omega} \text{tr}(\star d\omega_h) \wedge \text{tr}(\eta_h) 
$$

- $\omega$ and $\eta$ are both 1-forms
- $ \text{tr}(\star d\omega_h) $ 
- Exterior Derivative of $\omega$ is the curl, transforms 1-form $\omega$ to a 2-form.
- Hodge Star operator $\star$ transforms 2-form $\omega$ to a n-k or here a 0-form.
- The wedge product $\wedge$ of a 0-form (curl of $\omega$) and the trace of a 1-form 
  $\text{tr}(\eta)$ is simply a multiplication of a scalar by a tangential component.

WT: does Trace() give the tangential part or n times the tangential part?
valu that shozuld appear (n x curl(u)) * tangentialpart of v
can be rewritten -> curl(u) * ( v_tan x n)
2 ways: 1. .Trace() but make sure one knows what .Trace() does, whats the result? WT not entirely sure
2. just write what you see in n x curl(u) * v

In [7]:
t = specialcf.tangential(2)
a += curl(omega_HCurl) * eta_HCurl.Trace() * t * ds(skeleton=True, definedon = geo.Boundaries('default'))
#a += 

$$
.. + \int_{\partial \Omega} \text{tr} \, \omega_h \wedge \text{tr} \, \star d\eta_h
$$

In [8]:
# Turns out the hard part was the "skeleton=True" part lol, now this SHOULD be easy.
a += omega_HCurl.Trace() * t * curl(eta_HCurl) * ds(skeleton=True, definedon = geo.Boundaries('default'))

$$
.. + \frac{C_w}{h} \langle \text{tr}\, \omega, \text{tr}\, \eta \rangle_{\partial \Omega}
$$

In [9]:
# Define constant C_w (set the appropriate value for your problem)
C_w = 1000  # starting value, adjust as needed
h_min = min(Integrate(1, geo, BND, element_wise=True))

# Define the bilinear form for this boundary term
#penaltyTerm = BilinearForm(prodSpc)

# Boundary integral for the stabilization/penalty term
a += (C_w / h_min) * omega_HCurl.Trace() * eta_HCurl.Trace() * ds(skeleton=True, definedon = geo.Boundaries('default'))

manufactured solution?? lets try..
- This is the same solution Jay used in the vector laplacian publication
- I think they used this one because it vanishes on the boundary of the unit square
- which is very good  because I dont really get how to incorporate non-zero manufactured solutions in all the boundary terms.    


$$
\omega_{\text{exact}} = \begin{pmatrix} \sin(\pi x) \sin(\pi y) \\ \sin(\pi x) \sin(\pi y) \end{pmatrix}
$$

Need Hodge Laplace for it as well. first term of it:
- Exterior Coderivative of a 1-form in 2D: Divergence -> get a 0-form
- Taking the Exterior Derivative of a 0-form is equivalent to taking the gradient

Second Term:
- Exterior Derivative for 1-forms in 2D: Curl -> get a 2-form
- Taking the Exterior Coderivative of this 2-form formally gives a 1-form..
- Psuedovector operation <-> convention of plus and minus??

$$
\Delta_{\text{H}}\omega = (d \delta \omega + \delta d \omega) = \text{curl}(\text{rot}(\omega)) - \text{grad}(\text{div}(\omega)) = \text{f}
$$

Let's try to build the right hand sind with this manufactured solution!

WT: "if you program it smartly you only have to code 2 version bcs the dim wont matter!"

In [ ]:
omega_exact = CF( (sin(pi*x)*sin(pi*y), sin(pi*x)*sin(pi*y)))
delta_omega = -omega_exact[0].Diff(x)-omega_exact[1].Diff(y)
d_delta_omega_x = delta_omega.Diff(x)
d_delta_omega_x = delta_omega.Diff(y)
d_delta_omega = CF((d_delta_omega_x, d_delta_omega_x))

# Compute the exterior derivative -> curl rot (omega) in 2D
d_omega = omega_exact[1].Diff(x) - omega_exact[0].Diff(y)
delta_d_omega_x = d_omega.Diff(y)
delta_d_omega_y = - d_omega.Diff(x)
delta_d_omega = CF((delta_d_omega_x, delta_d_omega_y))
#delta_d_omega = CF((0, 0))

f_rhs = delta_d_omega + d_delta_omega
f = LinearForm(prodSpc)
f += f_rhs * eta_HCurl * dx

a.Assemble()
f.Assemble()


In [ ]:
import matplotlib.pyplot as plt
import scipy.sparse as sp
import numpy as np

rows,cols,vals = a.mat.COO()
A = sp.csr_matrix((vals,(rows,cols)))
plt.spy(A)
plt.show()
print("Matrix dimensions:", a.mat.height, "x", a.mat.width)
cond_nr = np.linalg.cond(A.todense())
print("Matrix condition number: ", cond_nr)

In [ ]:
sol = GridFunction(prodSpc)
res = f.vec-a.mat * sol.vec
inv = a.mat.Inverse(freedofs=prodSpc.FreeDofs(), inverse="pardiso")
sol.vec.data += inv * res
print(Norm(res))

In [ ]:
gf_omega , gf_sigma = sol.components

print("L2 Error:", sqrt(Integrate((gf_omega - omega_exact)**2, geo)))
print(gf_omega.vec)
Draw(gf_omega)

Thank you for using NGSolve
